# 잠재디리클레할당(LDA, Latent Dirichlet Allocation)

## 환경준비

In [1]:
!pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6 MB 6.3 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=36499303a86f7b0969bdc58c6436de2e441ab6b1d4c09b409084624582995939
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

##1)  sklearn 활용

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

#뉴스 다운로드 및 전처리
def get_news(apply_split=True) :
  #20newsgroup 다운로드
  dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
  documents = dataset.data

  news_df = pd.DataFrame({'document':documents})
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") # 특수 문자 제거
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) # 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())# 전체 단어에 대한 소문자 변환
  tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화

  stop_words = stopwords.words('english') # NLTK 불용어 조회

  if apply_split :
    return tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
  else :
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
#공백으로 토큰 분리
def my_tokenizer(text):
    return text.split()

tokenized_docs = get_news(False)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

    
tfidf_vect = TfidfVectorizer(tokenizer = my_tokenizer)
tfidf = tfidf_vect.fit_transform(tokenized_docs)
lda = LatentDirichletAllocation(n_components=20, 
                                max_iter=20, 
                                learning_method='online', 
                                random_state=100)

lda_output = lda.fit_transform(tfidf)

\

In [5]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/l

##2) gensim 활용

In [6]:
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

tokenized_docs = get_news()
id2word = corpora.Dictionary(tokenized_docs)
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 20
lda = LdaModel(corpus=corpus_TFIDF,
                    id2word=id2word,
                    num_topics=n, 
                    random_state=100)

for t in lda.print_topics() :
  print(t)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarn

(0, '0.003*"people" + 0.002*"would" + 0.002*"think" + 0.002*"government" + 0.002*"jesus" + 0.002*"good" + 0.002*"right" + 0.002*"know" + 0.002*"well" + 0.002*"really"')
(1, '0.002*"yankees" + 0.001*"catbyte" + 0.001*"dtmedin" + 0.001*"ingr" + 0.001*"alomar" + 0.001*"nrhj" + 0.001*"corp" + 0.001*"gilmour" + 0.001*"incredibly" + 0.001*"backed"')
(2, '0.002*"phillies" + 0.002*"hawks" + 0.002*"tiff" + 0.001*"candida" + 0.001*"preferred" + 0.001*"tapes" + 0.001*"test" + 0.001*"patent" + 0.001*"exhaust" + 0.001*"diet"')
(3, '0.002*"studio" + 0.002*"offer" + 0.002*"shipping" + 0.001*"polygon" + 0.001*"backup" + 0.001*"upgraded" + 0.001*"ultrix" + 0.001*"diseases" + 0.001*"floppies" + 0.001*"amps"')
(4, '0.002*"israel" + 0.002*"would" + 0.002*"said" + 0.002*"people" + 0.002*"like" + 0.002*"many" + 0.002*"israeli" + 0.002*"think" + 0.001*"know" + 0.001*"even"')
(5, '0.002*"objective" + 0.002*"morality" + 0.002*"space" + 0.002*"president" + 0.001*"score" + 0.001*"orbit" + 0.001*"bible" + 0.001*"

In [7]:
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

In [8]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TFIDF, id2word, mds='tsne')
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  FutureWarning,
